## Commencer a jouer
- import

---

In [5]:
import sys

In [6]:
#sys.executable

In [7]:
#sys.path
#Check Readme.md for detail sys.path.append(/users/....../bin/python3/jwojfoi )

In [8]:
import tensorflow as tf
import numpy as np
from vizdoom import *

import random
import time
from skimage import transform

from collections import deque
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [21]:
# On Créer la env
def creer_env():
    game = DoomGame()
    
    # Lire the setting et Initial
    game.load_config('/Users/johnny_hung/.ja_reinforecement_learning_lab/lib/python3.6/site-packages/vizdoom/scenarios/basic.cfg')
    game.set_doom_scenario_path('/Users/johnny_hung/.ja_reinforecement_learning_lab/lib/python3.6/site-packages/vizdoom/scenarios/basic.wad')
    game.init()
    
    # Les actions
    left  = [1,0,0]
    right = [0,1,0]
    shoot = [0,0,1]
    possible_actions = [left,right,shoot]
    
    return game, possible_actions

def test_env():
    game = DoomGame()
    
    # Lire the setting et Initial
    game.load_config('/Users/johnny_hung/.ja_reinforecement_learning_lab/lib/python3.6/site-packages/vizdoom/scenarios/basic.cfg')
    game.set_doom_scenario_path('/Users/johnny_hung/.ja_reinforecement_learning_lab/lib/python3.6/site-packages/vizdoom/scenarios/basic.wad')
    game.init()
    
    # Les actions
    left  = [1,0,0]
    right = [0,1,0]
    shoot = [0,0,1]
    possible_actions = [left,right,shoot]
    
    episodes = 3
    for i in range(episodes):
        print("Start new episode !!!!!!!!!!!!!!!!!!!!!!!!!!!")
        game.new_episode()
        while not game.is_episode_finished():
            state  = game.get_state()
            img    = state.screen_buffer
            misc   = state.game_variables
            action = random.choice(possible_actions)
            print("action = ",action)
            reward = game.make_action(action)
            print("         reward:", reward)
            time.sleep(0.02)
        print("Result: ",game.get_total_reward())
        time.sleep(2)
    game.close()
    

In [23]:
test_env()

Start new episode !!!!!!!!!!!!!!!!!!!!!!!!!!!
action =  [0, 1, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: 100.0
action =  [1, 0, 0]
         reward: -1.0
Result:  81.0
Start new episode !!!!!!!!!!!!!!!!!!!!!!!!!!!
action =  [0, 0, 1]
         reward: -1.0
action =  [

action =  [0, 1, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [1, 0, 0]
         reward: -6.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [1, 0, 0]
         rewar

         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -6.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0

action =  [0, 0, 1]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 0, 1]
         reward: -6.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [1, 0, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 1, 0]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
action =  [0, 0, 1]
         reward: -1.0
Result:  -390.0


In [24]:
game, possible_actions = creer_env()

In [ ]:
def preprocess_frame(frame):
    # Remove the root (useless information)
    cropped_frame = frame[30:-10,30:-30]
    # Normalize Pixel Values
    normalized_frame = cropped_frame/255.0
    # Resize
    preprocessed_frame = transform.resize(normalized_frame,[84,84])
    return preprocessed_frame

In [ ]:
frame_stack_size = 4

stacked_frames = deque( [np.zeros((84,84, dtype=np.int)) for i in range(frame_stack_size)], maxlen=4)

def stack_frames(stacked_frames, state, is_new_episode):
    # preprocess frame
    frame = preprocess_frame(state)